In [243]:
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf=myConf)\
        .getOrCreate()

### S.4.1 schema 생성하기

DataFrame은 데이터 모델 schema를 정의하고, 각 컬럼의 명칭과 데이터타입을 정해야 한다.

In [166]:
myList=[('1','kim, js', 170),
        ('1','lee, sm', 175),
        ('2','lim, yg',180),
        ('2','lee', 170)]
myDf=spark.cqreateDataFrame(myList)
myDf.columns

['_1', '_2', '_3']

In [167]:
myDf.printSchema()
print (myDf.take(3))

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: long (nullable = true)

[Row(_1='1', _2='kim, js', _3=170), Row(_1='1', _2='lee, sm', _3=175), Row(_1='2', _2='lim, yg', _3=180)]


#### 컬럼명 설정

앞서 컬럼 Column을 정의하지 않고 DataFrame을 생성하였는데, 이번에는 **컬럼을 정해서** 생성하자.
**```createDataFrame()```** 함수에 **인자로 컬럼명을 리스트 ```['year','name','height']```로** 정해준다.

In [179]:
cols = ['year','name','height']
_myDf = spark.createDataFrame(myList, cols)
_myDf.show()
print (_myDf.take(1))

+----+-------+------+
|year|   name|height|
+----+-------+------+
|   1|kim, js|   170|
|   1|lee, sm|   175|
|   2|lim, yg|   180|
|   2|    lee|   170|
+----+-------+------+

[Row(year='1', name='kim, js', height=170)]


In [23]:
names = ["kim","lee","lee","lim"]
items = ["espresso","latte","americano","affocato","long black","macciato"]

In [177]:
#아래 해석 총 100개를 뽑는데 위에 스키마 크기만큼 계속 할당시킴 

coffeeDf = spark.createDataFrame([(names[i%4], items[i%6]) for i in range(100)],\
                           ["name","coffee"])
coffeeDf.take(1)

[Row(name='kim', coffee='espresso')]

In [173]:
coffeeDf.printSchema()
coffeeDf.show(100)

root
 |-- name: string (nullable = true)
 |-- coffee: string (nullable = true)

+----+----------+
|name|    coffee|
+----+----------+
| kim|  espresso|
| lee|     latte|
| lee| americano|
| lim|  affocato|
| kim|long black|
| lee|  macciato|
| lee|  espresso|
| lim|     latte|
| kim| americano|
| lee|  affocato|
| lee|long black|
| lim|  macciato|
| kim|  espresso|
| lee|     latte|
| lee| americano|
| lim|  affocato|
| kim|long black|
| lee|  macciato|
| lee|  espresso|
| lim|     latte|
| kim| americano|
| lee|  affocato|
| lee|long black|
| lim|  macciato|
| kim|  espresso|
| lee|     latte|
| lee| americano|
| lim|  affocato|
| kim|long black|
| lee|  macciato|
| lee|  espresso|
| lim|     latte|
| kim| americano|
| lee|  affocato|
| lee|long black|
| lim|  macciato|
| kim|  espresso|
| lee|     latte|
| lee| americano|
| lim|  affocato|
| kim|long black|
| lee|  macciato|
| lee|  espresso|
| lim|     latte|
| kim| americano|
| lee|  affocato|
| lee|long black|
| lim|  macciato|
| 

 # Row 객체를 사용해서 생성 후 dataframe 생성

* Row 생성
**Row**를 사용해 보자.
Row는 **이름(Column)이 붙여진 행**으로 **관계형데이터베이스 레코드 Record**와 같다. 
속성 명은 'year', 'name', 'height'로 명명한다.

row는 어떠한 카테고리로 나눌지 결정해준다.

In [27]:
from pyspark.sql import Row
#row 생성
Person = Row('year','name','height')

#해당 row 형식에 맞게 데이터를 젛어줘서 schema 를 생성해라 
myRows = [Person('1','lee, sm', 175),
          Person('1','lee, sm', 175),
          Person('2','lim, yg',180),
          Person('2','lee',170)]

In [180]:
myDf = spark.createDataFrame(myRows)
myDf.printSchema()
myDf.show()

root
 |-- year: string (nullable = true)
 |-- name: string (nullable = true)
 |-- height: long (nullable = true)

+----+-------+------+
|year|   name|height|
+----+-------+------+
|   1|kim, js|   170|
|   1|lee, sm|   175|
|   2|lim, yg|   180|
|   2|    lee|   170|
+----+-------+------+



# schema를 정의하고 생성 후 dataFrame 에 넣기

모델 schema를 정하고, **데이터 타잎**을 정의해 DataFrame을 생성해 본다.
**```StructType```**으로 구조체를 선언하고, 컬럼에 대해 **```StructField```**를 설정한다.
* **컬럼**의 명칭
* 앞서 소개했던 **데이터 타잎**
* 마지막은 **NULL**이 허용되는지 여부

```python
StructType([
    StructField(컬럼명, StringType(), True),
    ...
])
```

In [37]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
mySchema=StructType([
    StructField("year", StringType(), True),
    StructField("name", StringType(), True),
    StructField("height", IntegerType(), True)
])

In [40]:
myDf=spark.createDataFrame(myRows, mySchema)
myDf.printSchema()
myDf.take(1)

root
 |-- year: string (nullable = true)
 |-- name: string (nullable = true)
 |-- height: integer (nullable = true)



[Row(year='1', name='kim, js', height=170)]

# S.4.2 RDD에서 생성하기

RDD는 schema가 정해지지 않은 비구조적 데이터이다.
이와 같이 **schema를 정의하지 않으면, Spark는 schema를 유추**하게 된다.


In [56]:
#ex 1)

In [185]:
myList=[('1','kim, js',170), 
        ('1','lee, sm', 175), 
        ('2','lim, yg',180), 
        ('2','lee',170)]
myrdd = spark.sparkContext.parallelize(myList)
mydf = myrdd.toDF()
mydf.show()

+---+-------+---+
| _1|     _2| _3|
+---+-------+---+
|  1|kim, js|170|
|  1|lee, sm|175|
|  2|lim, yg|180|
|  2|    lee|170|
+---+-------+---+



### row 사용하여 행생성
 

In [181]:
#예제 2

In [194]:
from pyspark.sql import Row
_myrdd = myrdd.map(lambda x : Row(year = int(x[0]), name = x[1], height = x[2]))
real = spark.createDataFrame(_myrdd)
real.printSchema()

root
 |-- year: long (nullable = true)
 |-- name: string (nullable = true)
 |-- height: long (nullable = true)



## S.4.3 Pandas


### Dataframe을 Pandas로 변환


In [197]:
mydf.toPandas()


,_1,_2,_3
0,1,"kim, js",170
1,1,"lee, sm",175
2,2,"lim, yg",180
3,2,lee,170


### Pandas에서 csv 파일로 내보내기

In [199]:
import os
mydf.write.format('com.databricks.spark.csv').save(os.path.join('data','dd.csv'))

In [201]:
!ls -l data/dd.csv/

total 8
-rw-r--r--  1 nadonghyeon  staff   0 Oct 22 18:26 _SUCCESS
-rw-r--r--  1 nadonghyeon  staff  58 Oct 22 18:26 part-00000-d60b15c3-0ca3-4ddb-8494-01357c16d56b-c000.csv


# Pandas에서 컬럼을 생성,삭제 해보자.
recode - 현재 변수 값을 다시 줄 경우

나라변 국제전화코드는 Japan: 81, South Korea: 82, Hong Kong: 852, Australia: 61을 사용한다

In [69]:
#json 형식으로 만듬
import pandas as pd
icc = pd.DataFrame( { 'country': ['South Korea','Japan','Hong Kong'],'codes': [81, 82, 852] })

In [68]:
icc

,country,codes
0,South Korea,81
1,Japan,82
2,Hong Kong,852


### S.4.4 csv 파일에서 생성

#### RDD에서 DataFrame

앞서 RDD에서 읽었던 csv파일을 다시 읽어보자.

```sparkContext.textFile()``` 함수로 읽은 파일은 RDD이다.

In [74]:
# strip => 앞 뒤에 관련 없는 단어가 들어오는거 방지하기 위해
# ex ) ad. 와 ad를 같은 단어로 인식한다. 따라서 strip()을 하면서 단어 제외시켜라

In [210]:
from pyspark.sql import Row
cfile= os.path.join("data", "ds_spark_2cols.csv")
lines = spark.sparkContext.textFile(cfile)
_col12 = line.map(lambda x: x.split(','))
col12 = new.map(lambda x: Row(col1 = int(x[0]), col2 = int(x[1])))
_myDf = spark.createDataFrame(col12)
_myDf.printSchema()
_myDf.show()

root
 |-- col1: long (nullable = true)
 |-- col2: long (nullable = true)

+----+----+
|col1|col2|
+----+----+
|  35|   2|
|  40|  27|
|  12|  38|
|  15|  31|
|  21|   1|
|  14|  19|
|  46|   1|
|  10|  34|
|  28|   3|
|  48|   1|
|  16|   2|
|  30|   3|
|  32|   2|
|  48|   1|
|  31|   2|
|  22|   1|
|  12|   3|
|  39|  29|
|  19|  37|
|  25|   2|
+----+----+



#### DataFrame으로 직접 읽기

format().load() 또는 csv() 함수로 csv 파일을 읽어서 DataFrame을 만들 수 있다.

In [78]:
%%writefile data/ds_spark.csv
1,2,3,4
11,22,33,44
111,222,333,444

Overwriting data/ds_spark.csv


In [209]:
df = spark\
        .read\
        .format('com.databricks.spark.csv')\
        .options(header='true', inferschema='true', delimiter=',')\
        .load(os.path.join('data','ds_spark.csv'))
df.show()

+---+---+---+---+
|  1|  2|  3|  4|
+---+---+---+---+
| 11| 22| 33| 44|
|111|222|333|444|
+---+---+---+---+



#### csv

또는 
```csv("path")```로 직접 DataFrame으로 읽을 수 있다.

In [211]:
df = spark\
        .read\
        .options(header='true', inferschema='true', delimiter=',')\
        .csv(os.path.join('data','ds_spark.csv'))
df.show()

+---+---+---+---+
|  1|  2|  3|  4|
+---+---+---+---+
| 11| 22| 33| 44|
|111|222|333|444|
+---+---+---+---+



### S.4.5 tsv 파일 읽기 (제대로 된 예제)

tsv (tab-separated values)는 **Tab으로 분리된 파일**을 말한다.
'\t'이 포함되어 있는 경우, 혹시 string으로 데이터타잎을 설정하기도 한다 (과거 Spark 버전에서)

In [114]:
#예제1

# 이거 다시 푸어

In [87]:
# %load data/ds_spark_heightweight.txt
1	65.78	112.99
2	71.52	136.49
3	69.40	153.03
4	68.22	142.34
5	67.79	144.30
6	68.70	123.30
7	69.80	141.49
8	70.01	136.46
9	67.90	112.37
10	66.78	120.67
11	66.49	127.45
12	67.62	114.14
13	68.30	125.61
14	67.12	122.46
15	68.28	116.09
16	71.09	140.00
17	66.46	129.50
18	68.65	142.97
19	71.23	137.90
20	67.13	124.04
21	67.83	141.28
22	68.88	143.54
23	63.48	97.90
24	68.42	129.50
25	67.63	141.85
26	67.21	129.72
27	70.84	142.42
28	67.49	131.55
29	66.53	108.33
30	65.44	113.89
31	69.52	103.30
32	65.81	120.75
33	67.82	125.79
34	70.60	136.22
35	71.80	140.10
36	69.21	128.75
37	66.80	141.80
38	67.66	121.23
39	67.81	131.35
40	64.05	106.71
41	68.57	124.36
42	65.18	124.86
43	69.66	139.67
44	67.97	137.37
45	65.98	106.45
46	68.67	128.76
47	66.88	145.68
48	67.70	116.82
49	69.82	143.62
50	69.09	134.93

SyntaxError: invalid syntax (<ipython-input-87-6ffbf930edea>, line 2)

경로 'data','ds_spark_heightweight.txt'

_rdd = rdd.map(lambda x : [float(x) for x in x.split('\t')]) 이거 주의

In [230]:
from pyspark.sql.types import *

file = os.path.join('data','ds_spark_heightweight.txt')
_tRdd= spark.sparkContext.textFile(file)
tRdd = _tRdd.map(lambda x : [float(x) for x in x.split('\t')])
tDfNamed = spark.createDataFrame(tRdd, ['value','weight','height'])
tDfNamed.show()

+-----+------+------+
|value|weight|height|
+-----+------+------+
|  1.0| 65.78|112.99|
|  2.0| 71.52|136.49|
|  3.0|  69.4|153.03|
|  4.0| 68.22|142.34|
|  5.0| 67.79| 144.3|
|  6.0|  68.7| 123.3|
|  7.0|  69.8|141.49|
|  8.0| 70.01|136.46|
|  9.0|  67.9|112.37|
| 10.0| 66.78|120.67|
| 11.0| 66.49|127.45|
| 12.0| 67.62|114.14|
| 13.0|  68.3|125.61|
| 14.0| 67.12|122.46|
| 15.0| 68.28|116.09|
| 16.0| 71.09| 140.0|
| 17.0| 66.46| 129.5|
| 18.0| 68.65|142.97|
| 19.0| 71.23| 137.9|
| 20.0| 67.13|124.04|
+-----+------+------+
only showing top 20 rows



## text 함수를 이용해 파일 읽음

In [241]:
from pyspark.sql.functions import split

tDftxt = spark.read.text(os.path.join('data','ds_spark_heightweight.txt'))
split = split(tDftxt['value'],'\t')
tDftxt = tDftxt.withColumn('weight',split.getItem(1))
tDftxt = tDftxt.withColumn('height',split.getItem(2))
tDftxt.show()


+---------------+------+------+
|          value|weight|height|
+---------------+------+------+
| 1	65.78	112.99| 65.78|112.99|
| 2	71.52	136.49| 71.52|136.49|
| 3	69.40	153.03| 69.40|153.03|
| 4	68.22	142.34| 68.22|142.34|
| 5	67.79	144.30| 67.79|144.30|
| 6	68.70	123.30| 68.70|123.30|
| 7	69.80	141.49| 69.80|141.49|
| 8	70.01	136.46| 70.01|136.46|
| 9	67.90	112.37| 67.90|112.37|
|10	66.78	120.67| 66.78|120.67|
|11	66.49	127.45| 66.49|127.45|
|12	67.62	114.14| 67.62|114.14|
|13	68.30	125.61| 68.30|125.61|
|14	67.12	122.46| 67.12|122.46|
|15	68.28	116.09| 68.28|116.09|
|16	71.09	140.00| 71.09|140.00|
|17	66.46	129.50| 66.46|129.50|
|18	68.65	142.97| 68.65|142.97|
|19	71.23	137.90| 71.23|137.90|
|20	67.13	124.04| 67.13|124.04|
+---------------+------+------+
only showing top 20 rows



In [242]:
split.getItem(1)

Column<'split(value, 	, -1)[1]'>

## csv 함수로 tsv 읽기

In [142]:
tDf = spark\
    .read\
    .options(header='false', inferschema='false', delimiter='\t')\
    .csv(os.path.join('data', 'ds_spark_heightweight.txt'))
tDf.show()

+---+-----+------+
|_c0|  _c1|   _c2|
+---+-----+------+
|  1|65.78|112.99|
|  2|71.52|136.49|
|  3|69.40|153.03|
|  4|68.22|142.34|
|  5|67.79|144.30|
|  6|68.70|123.30|
|  7|69.80|141.49|
|  8|70.01|136.46|
|  9|67.90|112.37|
| 10|66.78|120.67|
| 11|66.49|127.45|
| 12|67.62|114.14|
| 13|68.30|125.61|
| 14|67.12|122.46|
| 15|68.28|116.09|
| 16|71.09|140.00|
| 17|66.46|129.50|
| 18|68.65|142.97|
| 19|71.23|137.90|
| 20|67.13|124.04|
+---+-----+------+
only showing top 20 rows



### S.4.6 JSON 파일에서 생성

#### JSON
JSON은 JavaScript Object Notation, 즉 자바스크립트에서 사용되는 표기법. 사람이 읽을 수 있는 텍스트로 표기하며, key-value 쌍으로 되어 있다.
현재 널리 쓰이고 있어 XML 대용으로 널리 쓰이고 있다.
Spark example 폴더에 있는 ```people.json``` JSON 파일이다.

```python
{"name":"Michael"}
{"name":"Andy", "age":30}
{"name":"Justin", "age":19}
```

아래 인스타그램 데이터는 되지를 않음 

In [161]:
%%writefile src/ds_twitter_seoul_3.json



{"contributors": null, 
 "truncated": false, 
 "text": "RT @soompi: #SEVENTEEN’s Mingyu, Jin Se Yeon, And Leeteuk To MC For 2016 Super Seoul Dream Concert \nhttps://t.co/1XRSaRBbE0 https://t.co/fi…", 
 "is_quote_status": false, "in_reply_to_status_id": null, "id": 801657325836763136, "favorite_count": 0, 
 "entities": {"symbols": [], "user_mentions": [{"id": 17659206, "indices": [3, 10], "id_str": "17659206", "screen_name": "soompi", "name": "Soompi"}], 
              "hashtags": [{"indices": [12, 22], "text": "SEVENTEEN"}], 
              "urls": [{"url": "https://t.co/1XRSaRBbE0", "indices": [100, 123], "expanded_url": "http://www.soompi.com/2016/11/20/seventeens-mingyu-jin-se-yeon-leeteuk-mc-dream-concert/", "display_url": "soompi.com/2016/11/20/sev…"}]}, 
 "retweeted": false, "coordinates": null, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "in_reply_to_screen_name": null, "in_reply_to_user_id": null, "retweet_count": 1487, "id_str": "801657325836763136", "favorited": false, 
 "retweeted_status": {"contributors": null, "truncated": false, "text": "#SEVENTEEN’s Mingyu, Jin Se Yeon, And Leeteuk To MC For 2016 Super Seoul Dream Concert \nhttps://t.co/1XRSaRBbE0 https://t.co/fifXHpF8or", "is_quote_status": false, 
                      "in_reply_to_status_id": null, "id": 800593781586132993, "favorite_count": 1649, "entities": {"symbols": [], "user_mentions": [], "hashtags": [{"indices": [0, 10], "text": "SEVENTEEN"}], 
                                                                                                                    "urls": [{"url": "https://t.co/1XRSaRBbE0", "indices": [88, 111], "expanded_url": "http://www.soompi.com/2016/11/20/seventeens-mingyu-jin-se-yeon-leeteuk-mc-dream-concert/", "display_url": "soompi.com/2016/11/20/sev…"}], 
                                                                                                                    "media": [{"expanded_url": "https://twitter.com/soompi/status/800593781586132993/photo/1", 
                                                                                                                               "display_url": "pic.twitter.com/fifXHpF8or", "url": "https://t.co/fifXHpF8or", 
                                                                                                                               "media_url_https": "https://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg", "id_str": "800593115165798400", 
                                                                                                                               "sizes": {"small": {"h": 382, "resize": "fit", "w": 680}, "large": {"h": 449, "resize": "fit", "w": 800}, 
                                                                                                                                "medium": {"h": 449, "resize": "fit", "w": 800}, "thumb": {"h": 150, "resize": "crop", "w": 150}}, 
                                                                                                                               "indices": [112, 135], "type": "photo", "id": 800593115165798400, "media_url": "http://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg"}]}, 
                      "retweeted": false, "coordinates": null, "source": "<a href=\"https://about.twitter.com/products/tweetdeck\" rel=\"nofollow\">TweetDeck</a>", "in_reply_to_screen_name": null, "in_reply_to_user_id": null, "retweet_count": 1487, "id_str": "800593781586132993", 
                      "favorited": false, "user": {"follow_request_sent": false, "has_extended_profile": true, "profile_use_background_image": true, "default_profile_image": false, "id": 17659206, "profile_background_image_url_https": "https://pbs.twimg.com/profile_background_images/699864769/1cdde0a85f5c0a994ae1fb06d545a5ec.png", "verified": true, "translator_type": "none", "profile_text_color": "999999", 
                                                   "profile_image_url_https": "https://pbs.twimg.com/profile_images/792117259489583104/4khJk3zz_normal.jpg", "profile_sidebar_fill_color": "000000", "entities": {"url": {"urls": [{"url": "http://t.co/3evT80UlR9", "indices": [0, 22], "expanded_url": "http://www.soompi.com", "display_url": "soompi.com"}]}, "description": {"urls": []}}, "followers_count": 987867, "profile_sidebar_border_color": "000000", "id_str": "17659206", "profile_background_color": "1E1E1E", "listed_count": 3982, "is_translation_enabled": true, "utc_offset": -28800, "statuses_count": 80038, "description": "The original K-pop community. We take gifs, OTPs, and reporting on your bias' fashion choices seriously. But not rumors. Ain't nobody got time for that.", "friends_count": 3532, "location": "Worldwide", "profile_link_color": "31B6F4", "profile_image_url": "http://pbs.twimg.com/profile_images/792117259489583104/4khJk3zz_normal.jpg", "following": false, "geo_enabled": false, "profile_banner_url": "https://pbs.twimg.com/profile_banners/17659206/1478803767", "profile_background_image_url": "http://pbs.twimg.com/profile_background_images/699864769/1cdde0a85f5c0a994ae1fb06d545a5ec.png", "screen_name": "soompi", "lang": "en", "profile_background_tile": true, "favourites_count": 1493, "name": "Soompi", "notifications": false, "url": "http://t.co/3evT80UlR9", "created_at": "Wed Nov 26 20:48:27 +0000 2008", "contributors_enabled": false, "time_zone": "Pacific Time (US & Canada)", "protected": false, "default_profile": false, "is_translator": false}, "geo": null, "in_reply_to_user_id_str": null, "possibly_sensitive": false, "lang": "en", "created_at": "Mon Nov 21 06:56:46 +0000 2016", "in_reply_to_status_id_str": null, "place": null, "extended_entities": {"media": [{"expanded_url": "https://twitter.com/soompi/status/800593781586132993/photo/1", "display_url": "pic.twitter.com/fifXHpF8or", "url": "https://t.co/fifXHpF8or", "media_url_https": "https://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg", "id_str": "800593115165798400", "sizes": {"small": {"h": 382, "resize": "fit", "w": 680}, "large": {"h": 449, "resize": "fit", "w": 800}, "medium": {"h": 449, "resize": "fit", "w": 800}, "thumb": {"h": 150, "resize": "crop", "w": 150}}, "indices": [112, 135], "type": "photo", "id": 800593115165798400, "media_url": "http://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg"}]}, "metadata": {"iso_language_code": "en", "result_type": "recent"}}, "user": {"follow_request_sent": false, "has_extended_profile": false, "profile_use_background_image": true, "default_profile_image": true, "id": 791090169818521600, "profile_background_image_url_https": null, "verified": false, "translator_type": "none", "profile_text_color": "333333", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_6_normal.png", "profile_sidebar_fill_color": "DDEEF6", "entities": {"description": {"urls": []}}, "followers_count": 0, "profile_sidebar_border_color": "C0DEED", "id_str": "791090169818521600", "profile_background_color": "F5F8FA", "listed_count": 0, "is_translation_enabled": false, "utc_offset": null, "statuses_count": 96, "description": "", "friends_count": 7, "location": "", "profile_link_color": "1DA1F2", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_6_normal.png", "following": false, "geo_enabled": false, "profile_background_image_url": null, "screen_name": "enriquesanq", "lang": "es", "profile_background_tile": false, "favourites_count": 161, "name": "Enrique santos", "notifications": false, "url": null, "created_at": "Wed Oct 26 01:32:49 +0000 2016", "contributors_enabled": false, "time_zone": null, "protected": false, "default_profile": true, "is_translator": false}, "geo": null, "in_reply_to_user_id_str": null, "possibly_sensitive": false, "lang": "en", "created_at": "Thu Nov 24 05:22:55 +0000 2016", "in_reply_to_status_id_str": null, "place": null, "metadata": {"iso_language_code": "en", "result_type": "recent"}}'

Overwriting src/ds_twitter_seoul_3.json


In [154]:
#json파일이라도 모두가 문자열 
# 여기에 의미를 부여한다 -> 파싱한다. 

In [156]:
import os
_jfname=os.path.join('src','ds_twitter_seoul_3.json')
with open(_jfname, 'rb') as f:
    data = f.readlines()

In [157]:
import json
data_json_str = json.loads(data[0])

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

# pandas 에서 트윗 읽기는 보지마 


# dataFrame 에서 json 데이터 읽기

In [160]:
jfile= os.path.join('src','ds_twitter_seoul_3.json')

tweetDf= spark.read.json(jfile)
tweetDf.printSchema()
tweetDf.count()
tweetDf.select('id', 'text').show(10)

root
 |-- _corrupt_record: string (nullable = true)

